In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import tensorflow
tensorflow.keras.__version__
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

In [2]:
#Reading the csv into pandas
data = pd.read_csv('csv_data/Herion Drug Data.csv')
data = data.fillna(0)
data.head()

,KEY,state,year,month,period,indicator,data_value,state_name,Date,unemployment_rate
0,HawaiiNovember2017,HI,2017.0,November,12 month-ending,Heroin (T40.1),0.0,Hawaii,11/1/2017,2.2
1,HawaiiOctober2017,HI,2017.0,October,12 month-ending,Heroin (T40.1),0.0,Hawaii,10/1/2017,2.2
2,HawaiiApril2018,HI,2018.0,April,12 month-ending,Heroin (T40.1),13.0,Hawaii,4/1/2018,2.3
3,HawaiiAugust2017,HI,2017.0,August,12 month-ending,Heroin (T40.1),0.0,Hawaii,8/1/2017,2.3
4,HawaiiDecember2017,HI,2017.0,December,12 month-ending,Heroin (T40.1),0.0,Hawaii,12/1/2017,2.3


In [3]:
#creating our lists
X = data[["year"]]
X.head()

,year
0,2017.0
1,2017.0
2,2018.0
3,2017.0
4,2017.0


In [4]:

y = data["unemployment_rate"]


In [5]:
#Creating labels
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

#encoded_y_train = encoded_y_train.astype(float)
#encoded_y_test = encoded_y_test.astype(float)
# Step 2: Convert encoded labels to one-hot-encoding
y_categorical = to_categorical(encoded_y)

In [6]:
#Splitting our training data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=100)
X_train = X_train.values
X_test = X_test.values
print(X_train.shape)
print(y)

(1249, 1)
0       2.2
1       2.2
2       2.3
3       2.3
4       2.3
       ... 
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
Name: unemployment_rate, Length: 1666, dtype: float64


In [7]:
#Scaling our data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(X_train.shape)

(1249, 1)


In [8]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=y_categorical.shape[1], activation='softmax'))

In [9]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(X_train_scaled.shape)
print(y_train.shape)

(1249, 1)
(1249, 53)


In [10]:
#Running the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=1
)

Epoch 1/60
40/40 [==============================] - 0s 773us/step - loss: 3.9289 - accuracy: 0.1177
Epoch 2/60
40/40 [==============================] - 0s 698us/step - loss: 3.8143 - accuracy: 0.1345
Epoch 3/60
40/40 [==============================] - 0s 723us/step - loss: 3.7080 - accuracy: 0.1449
Epoch 4/60
40/40 [==============================] - 0s 848us/step - loss: 3.5845 - accuracy: 0.1377
Epoch 5/60
40/40 [==============================] - 0s 748us/step - loss: 3.4532 - accuracy: 0.1449
Epoch 6/60
40/40 [==============================] - 0s 798us/step - loss: 3.3980 - accuracy: 0.1449
Epoch 7/60
40/40 [==============================] - 0s 748us/step - loss: 3.3805 - accuracy: 0.1449
Epoch 8/60
40/40 [==============================] - 0s 798us/step - loss: 3.3768 - accuracy: 0.1449
Epoch 9/60
40/40 [==============================] - 0s 748us/step - loss: 3.3744 - accuracy: 0.1369
Epoch 10/60
40/40 [==============================] - 0s 748us/step - loss: 3.3760 - accuracy: 0.1369

In [11]:
#Prediction Percentage
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

14/14 - 0s - loss: 3.3288 - accuracy: 0.1511
Normal Neural Network - Loss: 3.3287711143493652, Accuracy: 0.15107913315296173


In [12]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [13]:
#Model Labels
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0.  4.4 4.4 4.4 4.4]
Actual Labels: [array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.,

In [ ]:
.to_csv("csv_data/DrugData.csv",index=False)